# Assignment 1

Our decision variable:

$ x_{ijk}$ = Allocation of department i is assigned to a room type j on day k 

$ t_{i}$ = Target hours for department i 

$ d_{jk}$ = Duration of operation block for room j on day k

$ o_{i}$ = Overallocation of department i 

$ u_{i}$ = Undersallocation of department i subject to:

In [1]:
# Import gurobi and numpy
from gurobipy import *
import numpy as np

In [2]:
mod = Model()
dept = [1, 2, 3, 4, 5, 6]
OR = [1, 2, 3, 4, 5]
days = [1, 2, 3, 4, 5]
floors = [1, 2, 3]

Restricted license - for non-production use only - expires 2023-10-25


In [3]:
#defining X variables 
X = {}
for i in dept:
    for j in OR: 
        for k in days: 
            X[i,j,k] = mod.addVar(vtype = GRB.BINARY, name = "x_%d_%d_%d"% (i,j,k))

In [4]:
#defining duration variables
d = {}
for j in OR: 
    for k in days: 
        if j == 1: 
            d[j,k] = 9
        elif j in [2,3,4] and k in [1,2,3,4] :
            d[j,k] = 9
        elif j in [2,3,4] and k == 5:
            d[j,k] = 8
        elif j == 5 and k in [1,2,3,4]:
            d[j,k] = 7.5
        elif j == 5 and k == 5:
            d[j,k] = 6.5
d

{(1, 1): 9,
 (1, 2): 9,
 (1, 3): 9,
 (1, 4): 9,
 (1, 5): 9,
 (2, 1): 9,
 (2, 2): 9,
 (2, 3): 9,
 (2, 4): 9,
 (2, 5): 8,
 (3, 1): 9,
 (3, 2): 9,
 (3, 3): 9,
 (3, 4): 9,
 (3, 5): 8,
 (4, 1): 9,
 (4, 2): 9,
 (4, 3): 9,
 (4, 4): 9,
 (4, 5): 8,
 (5, 1): 7.5,
 (5, 2): 7.5,
 (5, 3): 7.5,
 (5, 4): 7.5,
 (5, 5): 6.5}

In [5]:
#defining target allocation variable 
Target_allocation = [48.4, 4.2, 25.3, 7.4, 5.3, 9.5]
total_hrs = 213.5
t = {}
for a in range(len(Target_allocation)):
    t[a+1] = round(Target_allocation[a] * total_hrs / 100, 2)
t

{1: 103.33, 2: 8.97, 3: 54.02, 4: 15.8, 5: 11.32, 6: 20.28}

In [6]:
F={}
for i in dept:
    for k in days:
        for l in floors:
            F[i,k,l] = mod.addVar(vtype = GRB.BINARY, name = "f_%d_%d_%d"% (i,k,l))

In [7]:
u = {}
for i in dept:
    u[i] = mod.addVar(vtype = GRB.CONTINUOUS, name = "y_%d"% (i))

In [8]:
o = {}
for i in dept:
    o[i] = mod.addVar(vtype = GRB.CONTINUOUS, name = "y_%d"% (i))

## Model Constraints

### Objective Function
Minimize $\sum_{u,t=1}^6 \frac{u_{i}}{t_{i}} $

### Constraints for Part A

#### Balance Constraint
$\sum_{j=1}^5 \sum_{k=1}^5 d_{jk} x_{ijk}+ u_{i} - o_{i} -t_{i} = 0 $

#### Room Constraint
$\sum_{l=1}^6 x_{ijk} <=1 $ for all i,k

#### Non-negativity Constraint
$u_{i} >=0 $

$o_{i}  >=0 $


### Additional Constraints of Part B

#### Floor Constraint

$\sum_{l=1}^3 F_{ikl} <=1 $ for all i,k

#### Floor/Room Combinations Constraint
$x_{i1k} + x_{i2k} <= 2 * F_{ik1} $ for all i,k

$x_{i3k} + x_{i4k} <= 2 * F_{ik2} $ for all i,k

$x_{i,5,k} <= F_{ik3} $ for all i,k

In [ ]:
#Under allocation non negativity constraint

In [9]:
#Balance Constraint 
for i in dept: 
        mod.addConstr(sum((X[i,j,k]*d[j,k]) for j in OR for k in days)+u[i]-o[i]-t[i] == 0)           

In [10]:
#Room Constraint 
for j in OR: 
    for k in days: 
        mod.addConstr(sum(X[i,j,k] for i in dept) <= 1)

In [11]:
# Floor & Floor/ Room constraints
for i in dept:
    for k in days:
        mod.addConstr(sum(F[i,k,l] for l in floors)<= 1)
        mod.addConstr(X[i,1,k]+ X[i,2,k] <= 2*F[i,k,1])
        mod.addConstr(X[i,3,k]+ X[i,4,k] <= 2*F[i,k,2])
        mod.addConstr(X[i,5,k] <= 2*F[i,k,3])

In [12]:
#objective function
mod.setObjective(sum(u[i]/t[i] for i in dept), GRB.MINIMIZE)           

In [13]:
mod.update()
mod.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 151 rows, 252 columns and 642 nonzeros
Model fingerprint: 0x9d2de466
Variable types: 12 continuous, 240 integer (240 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e-02, 1e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 6.0000000
Presolve removed 60 rows and 66 columns
Presolve time: 0.00s
Presolved: 91 rows, 186 columns, 516 nonzeros
Variable types: 0 continuous, 186 integer (180 binary)
Found heuristic solution: objective 5.9893361

Root relaxation: objective 1.386819e-01, 120 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.13868    0    5    5.98934    0.13868  97.7%     

In [14]:
mod.objVal

0.13868189296428918

In [16]:
import pandas as pd
for v in mod.getVars():
    print(v.varName, v.x)

x_1_1_1 -0.0
x_1_1_2 -0.0
x_1_1_3 -0.0
x_1_1_4 1.0
x_1_1_5 1.0
x_1_2_1 -0.0
x_1_2_2 -0.0
x_1_2_3 -0.0
x_1_2_4 1.0
x_1_2_5 1.0
x_1_3_1 1.0
x_1_3_2 1.0
x_1_3_3 1.0
x_1_3_4 -0.0
x_1_3_5 -0.0
x_1_4_1 1.0
x_1_4_2 1.0
x_1_4_3 1.0
x_1_4_4 -0.0
x_1_4_5 -0.0
x_1_5_1 -0.0
x_1_5_2 -0.0
x_1_5_3 -0.0
x_1_5_4 -0.0
x_1_5_5 -0.0
x_2_1_1 -0.0
x_2_1_2 0.0
x_2_1_3 0.0
x_2_1_4 -0.0
x_2_1_5 -0.0
x_2_2_1 -0.0
x_2_2_2 0.0
x_2_2_3 -0.0
x_2_2_4 -0.0
x_2_2_5 -0.0
x_2_3_1 -0.0
x_2_3_2 -0.0
x_2_3_3 -0.0
x_2_3_4 0.0
x_2_3_5 0.0
x_2_4_1 0.0
x_2_4_2 -0.0
x_2_4_3 0.0
x_2_4_4 1.0
x_2_4_5 -0.0
x_2_5_1 0.0
x_2_5_2 -0.0
x_2_5_3 -0.0
x_2_5_4 -0.0
x_2_5_5 0.0
x_3_1_1 1.0
x_3_1_2 -0.0
x_3_1_3 -0.0
x_3_1_4 -0.0
x_3_1_5 -0.0
x_3_2_1 1.0
x_3_2_2 -0.0
x_3_2_3 -0.0
x_3_2_4 -0.0
x_3_2_5 -0.0
x_3_3_1 -0.0
x_3_3_2 -0.0
x_3_3_3 -0.0
x_3_3_4 -0.0
x_3_3_5 1.0
x_3_4_1 -0.0
x_3_4_2 -0.0
x_3_4_3 -0.0
x_3_4_4 -0.0
x_3_4_5 1.0
x_3_5_1 -0.0
x_3_5_2 1.0
x_3_5_3 1.0
x_3_5_4 1.0
x_3_5_5 0.0
x_4_1_1 -0.0
x_4_1_2 -0.0
x_4_1_3 1.0
x_4_1_4 -0.0
x_

In [ ]:
OR = []
days = []
departments = []
days_OR = []
import pandas as pd
for v in mod.getVars():
    if v.x  == 1:
        departments.append(str(v.varName)[2:3])
        OR.append(str(v.varName)[4:5])
        days.append(str(v.varName)[6:7])
        days_OR.append([str(v.varName)[6:7], str(v.varName)[4:5]]) 

In [ ]:
df = pd.DataFrame()
allocation = pd.DataFrame()

In [ ]:
for i in range(1,6):
    for j in range(1,6):
        if([str(i), str(j)] in days_OR):
            index = days_OR.index([str(i), str(j)])
            df.loc[i,j] = departments[index]
            allocation.loc[i,j] = 1 * d[j,i]
        

In [ ]:
Assigned_allocation = [0]*7
for i in range(1,6):
    for j in range(1,6):
        #print(int(df.loc[i,j]), allocation.loc[i,j])
        Assigned_allocation[int(df.loc[i,j])] += allocation.loc[i,j]/213.5
        #print(temp)
        
Assigned_allocation = [round(num*100, 1) for num in Assigned_allocation]
Assigned_allocation = Assigned_allocation[1:7]

In [ ]:
Assigned_allocation

In [ ]:
Target_allocation

In [ ]:
d = {'Target_allocation':Target_allocation,'Assigned_allocation':Assigned_allocation}
Allocation = pd.DataFrame(d)
Allocation = pd.DataFrame(d, columns=['Target_allocation','Assigned_allocation'])

In [ ]:
Allocation['Difference'] = Allocation['Target_allocation'] - Allocation['Assigned_allocation']

In [ ]:
df.columns =['Main 1', 'Main 2', 'Main 3', 'Main 4', 'Main 5']
df.index =['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']

df = df.replace(to_replace =['1'], value ="General Surgery")
df = df.replace(to_replace =['2'], value ="Emergency")
df = df.replace(to_replace =['3'], value ="Neurosurgery")
df = df.replace(to_replace =['4'], value ="Opthamology")
df = df.replace(to_replace =['5'], value ="Oral Surgery")
df = df.replace(to_replace =['6'], value ="Otolaryngology")

In [ ]:
df

In [ ]:
Allocation

In [ ]:
Allocation['Difference'].abs().sum()